<a href="https://colab.research.google.com/github/variouscafe/variouscafe.github.io/blob/master/getting_started_with_openai_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OpenAI API(챗 완성) 사용해보기

In [ ]:
!pip install openai==0.28 # openai 라이브러리를 설치합니다.

In [ ]:
import os

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
import json

# 개발자가 연동을 하고 싶은 함수를 정의한다.
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
ret = get_current_weather('서울')

In [ ]:
print(ret)

{"location": "\uc11c\uc6b8", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}


In [ ]:
def get_gps():
    return '서울'

In [ ]:
# 함수 호출을 위한 dict

available_functions = {
    "get_current_weather": get_current_weather,
    "get_location_from_gps" : get_gps,
}

In [ ]:
ret = available_functions["get_current_weather"]('서울')

print(ret)

{"location": "\uc11c\uc6b8", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}


In [ ]:
ret = available_functions["get_location_from_gps"]()

print(ret)

서울


In [ ]:
def run_conversation():
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": "What's the weather in Seoul?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )

    print(response)

    response_message = response["choices"][0]["message"]

    # 함수 특정에 성공하면 "function_call" 키가 존재합니다.
    print(response_message)

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        # 함수 이름과 인자를 얻어온다.
        function_name = response_message["function_call"]["name"]
        function_args = json.loads(response_message["function_call"]["arguments"])

        # 함수를 호출해서 그 결과를 얻는다.
        function_response = available_functions[function_name](
            location = function_args.get("location"),
            unit = function_args.get("unit"),
        )

        # response_message : function_call, name, arguments
        # 함수 이름과 인자 정보를 담은 메시지를 추가한다.
        messages.append(response_message)  # extend conversation with assistant's reply

        # 함수 호출 결과를 메시지에 추가한다.
        messages.append(
            {
                "role": "function", # 역할 : 함수 호출 결과는 역할을 function으로 설정한다.
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("## messages ##")
        print(messages)
        print("## messages ##")

        second_response = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo-0613",
            messages = messages, # 사용자 요청사항, 함수 이름 및 인자, 함수 호출 결과
        )  # get a new response from GPT where it can see the function response

        return second_response

print(run_conversation())

{
  "id": "chatcmpl-97Fpng5Fo9qxmTXUDJz6IXH5SXaYE",
  "object": "chat.completion",
  "created": 1711516191,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Seoul\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 81,
    "completion_tokens": 17,
    "total_tokens": 98
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Seoul\"\n}"
  }
}
## messages ##
[{'role': 'user', 'content': "What's the weather in Seoul?"}, <OpenAIObject at 0x7a5dda5d5f80> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"locat

In [ ]:
def run_conversation(user_input_text):
    # Step 1: send the conversation and available functions to GPT
    messages = [{"role": "user", "content": user_input_text}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
        {
            "name": "get_location_from_gps",
            "description": "Get the current location",
            "parameters": {
                "type": "object",
                "properties": {
                    "unit": {"type": "string"},
                },
                "required": [],
            },
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  # auto is default, but we'll be explicit
    )

    print(response)

    response_message = response["choices"][0]["message"]

    print(response_message)

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        # 함수 이름과 함수 인자를 얻는다.
        function_name = response_message["function_call"]["name"]
        function_args = json.loads(response_message["function_call"]["arguments"])

        # 함수 호출
        # 어떤 함수가 호출되는 지 모르기 때문에, 즉 인자를 특정할 수 없다.
        function_response = available_functions[function_name](**function_args)

        # 함수 이름과 인자 정보 추가
        messages.append(response_message)  # extend conversation with assistant's reply

        # 함수 호출 결과 추가 (role : function)
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("## messages ##")
        print(messages)
        print("## messages ##")

        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )  # get a new response from GPT where it can see the function response
        return second_response

In [ ]:
message = run_conversation("현재 위치는 어디인가요?")

{
  "id": "chatcmpl-97FppXSJNv4lahKjJPVD10ydzoeB1",
  "object": "chat.completion",
  "created": 1711516193,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_location_from_gps",
          "arguments": "{}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 108,
    "completion_tokens": 9,
    "total_tokens": 117
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_location_from_gps",
    "arguments": "{}"
  }
}
## messages ##
[{'role': 'user', 'content': '현재 위치는 어디인가요?'}, <OpenAIObject at 0x7a5dda5e78d0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_location_from_gps",
    "arguments": "{}"
  }
}, {'role': 'function', 'name': 'get_location_from_gps', 'content': '서

In [ ]:
message.choices[0].message.content

'현재 위치는 서울입니다.'

In [ ]:
message = run_conversation("서울의 날씨는요?")

{
  "id": "chatcmpl-97FpqVvPljQAy7azRxmFzk6AZklma",
  "object": "chat.completion",
  "created": 1711516194,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Seoul\"\n}"
        }
      },
      "logprobs": null,
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 109,
    "completion_tokens": 17,
    "total_tokens": 126
  },
  "system_fingerprint": null
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Seoul\"\n}"
  }
}
## messages ##
[{'role': 'user', 'content': '서울의 날씨는요?'}, <OpenAIObject at 0x7a5dda5e4c20> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Seoul\"\

In [ ]:
message.choices[0].message.content

'서울의 현재 온도는 72°F이며, 날씨는 맑고 바람이 강합니다.'